# Import libraries

In [ ]:
import warnings
warnings.filterwarnings( 'ignore' )

In [ ]:
# Built-in libraries
#
import math
import time
import json
import numpy    as np
import pandas   as pd
from   tqdm     import tqdm
from   datetime import datetime


# Visualization libraries
#
import matplotlib.pyplot as plt


# Sklearn
#
from sklearn               import metrics
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Sklearn-Optimization
#
import skopt
import skopt.plots


# hiplot
#
import hiplot as hip


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# Tensorflow library
#
import tensorflow 
from   tensorflow.keras.metrics                 import *
from   tensorflow.keras.optimizers              import *
from   tensorflow.keras.callbacks               import EarlyStopping
from   tensorflow.keras.callbacks               import ModelCheckpoint
from   tensorflow.keras.callbacks               import ReduceLROnPlateau


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# NBeats library
#
from utils.NBeats_Keras import NBeatsNet


# =-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
# User libraries
#
from utils.PerformanceMetrics import *

In [ ]:
from tensorflow.python.keras.utils.generic_utils import get_custom_objects
from tensorflow.keras.layers import Activation

def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

        
get_custom_objects().update({'swish': Activation(swish)})



# from tensorflow.keras.utils  import get_custom_objects

def gelu(x):
    return 0.5 * x * (1 + tensorflow.tanh(tensorflow.sqrt(2 / np.pi) * (x + 0.044715 * x**3)))
    # return tf.keras.backend.hard_sigmoid(1.702 * x) * x

get_custom_objects().update({'gelu':  Activation(gelu)})

# Parameters

In [ ]:
import random, os
seed = 42
    
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
tensorflow.random.set_seed(seed)

In [ ]:
# HPO parameters 
#
n_calls         = 100
n_random_starts =  10


# Parameters
#
filename        = '../metadata/7-kanal-1.csv'
#
Transformation  = True
Scaling         = 'Standard'
#
Lag        =  12
Horizon    =   4
#
epochs     =  300

## CUDA

In [ ]:
# try:
#     gpus = tensorflow.config.list_physical_devices('GPU')
#     if gpus:
#         try:
#             # Currently, memory growth needs to be the same across GPUs
#             for gpu in gpus:
#                 tensorflow.config.experimental.set_memory_growth(gpu, True)

#             logical_gpus = tensorflow.config.list_logical_devices('GPU')
#             print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

#         except RuntimeError as e:

#             # Memory growth must be set before GPUs have been initialized
#             print(e)
# except:
#     print('[INFO] Not GPU found')

# select GPU [0, 1] --> 1 for you
import os
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

# set memory growth
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# physical_devices = tf.config.list_physical_devices('GPU')
# if physical_devices:
#     tf.config.experimental.set_memory_growth(physical_devices[0], True)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    print('[INFO] GPU granted')
except:
    print('[INFO] GPU NOT granted')    
    # Invalid device or cannot modify virtual devices once initialized.
    pass

# Import data

## Loading data

In [ ]:
# Start timer
#
start = time.time()

# Load data
#
df = pd.read_csv( filename )

print('[INFO] Data imported')
print('[INFO] Time: %.2f seconds' % (time.time() - start))

df.head(3)

## Pre-processing data

In [ ]:
# Convert Date to 'datetime64'
#
df['Date'] = df['Date'].astype('datetime64')

# Set index
#
df.set_index('Date', inplace=True)


df = df.resample('5min').mean().interpolate()
# df = pd.DataFrame( df[['Temperature', 'Humidity', 'NO2', 'CO']] )

# The last feature is the target variable
df = pd.DataFrame( df[ ['NO2', 'CO'] ] )
# df = pd.DataFrame( df[ 'CO' ] )


df.head(3)

In [ ]:
targetSeries = df.columns[-1]

# Training/Testing sets

In [ ]:
idx = int( df.shape[0] * 0.8 )

df_train = df[ :idx ]
df_test  = df[ idx: ]

## Visualization 

In [ ]:
plt.figure( figsize = (20, 4) );
#
df_train[ targetSeries ].plot()
df_test[ targetSeries ].plot()
#
plt.legend(['Training', 'Testing'], fontsize = 12, frameon = False);
plt.ylabel( targetSeries, size = 12);
plt.xlabel( 'Time', size = 12);

## Fixing Lag

In [ ]:
df_test = pd.concat([df_train.iloc[-Lag:], df_test])

## Data transformation

In [ ]:
if (Transformation == True):
    
    print('[INFO] Data transformation applied')
    
    VALUE = max(df.min().min(), 1.0)
    
    df_train = np.log( df_train + VALUE)
    df_test  = np.log( df_test  + VALUE)
    
else:
    print('[INFO] No data transformation applied.')  

## Data scaling

In [ ]:
if (Scaling == 'MinMax'):
    print('[INFO] Scaling: MinMax')
    
    for feature in df.columns:
        if (feature == targetSeries): continue
        print('Feature: ', feature)        
        # Set scaler
        #
        scaler = MinMaxScaler()
        
        df_train[feature] = scaler.fit_transform( df_train[ feature ].to_numpy().reshape(-1,1) )
        df_test[feature]  = scaler.transform( df_test[ feature ].to_numpy().reshape(-1,1) )

        
    # Scaling of Target Series
    #
    scaler = MinMaxScaler()
    df_train[targetSeries] = scaler.fit_transform( df_train[ targetSeries ].to_numpy().reshape(-1,1) )
    df_test[targetSeries]  = scaler.transform( df_test[ targetSeries ].to_numpy().reshape(-1,1) )
            
elif (Scaling == 'Robust'):
    print('[INFO] Scaling: Robust')
    
    for feature in df.columns:
        if (feature == targetSeries): continue
        print('Feature: ', feature)        
        # Set scaler
        #
        scaler = RobustScaler()
        
        df_train[feature] = scaler.fit_transform( df_train[ feature ].to_numpy().reshape(-1,1) )
        df_test[feature]  = scaler.transform( df_test[ feature ].to_numpy().reshape(-1,1) )

        
    # Scaling of Target Series
    #
    scaler = RobustScaler()
    df_train[targetSeries] = scaler.fit_transform( df_train[ targetSeries ].to_numpy().reshape(-1,1) )
    df_test[targetSeries]  = scaler.transform( df_test[ targetSeries ].to_numpy().reshape(-1,1) )
        
elif (Scaling == 'Standard'):
    print('[INFO] Scaling: Standard')

    for feature in df.columns:
        if (feature == targetSeries): continue
        print('Feature: ', feature)
        # Set scaler
        #
        scaler = StandardScaler()
        
        df_train[feature] = scaler.fit_transform( df_train[ feature ].to_numpy().reshape(-1,1) )
        df_test[feature]  = scaler.transform( df_test[ feature ].to_numpy().reshape(-1,1) )

        
    # Scaling of Target Series
    #
    scaler = StandardScaler()
    df_train[targetSeries] = scaler.fit_transform( df_train[ targetSeries ].to_numpy().reshape(-1,1) )
    df_test[targetSeries]  = scaler.transform( df_test[ targetSeries ].to_numpy().reshape(-1,1) )
           
else:
    print('[WARNING] Unknown data scaling. Standar scaling was selected')   
    
    for feature in df.columns:
        if (feature == targetSeries): continue
        print('Feature: ', feature)
        # Set scaler
        #
        scaler = StandardScaler()
        
        df_train[feature] = scaler.fit_transform( df_train[ feature ].to_numpy().reshape(-1,1) )
        df_test[feature]  = scaler.transform( df_test[ feature ].to_numpy().reshape(-1,1) )

        
    # Scaling of Target Series
    #
    scaler = StandardScaler()
    df_train[targetSeries] = scaler.fit_transform( df_train[ targetSeries ].to_numpy().reshape(-1,1) )
    df_test[targetSeries]  = scaler.transform( df_test[ targetSeries ].to_numpy().reshape(-1,1) )  

## Create Training/Testing datasets

In [ ]:
def create_dataset(df = None, Lag = 1, Horizon = 12, targetSeries = None, includeLagData = False, includeDate = False, overlap = 1):
    
    if (targetSeries is None):
        targetSeries = df.columns[-1]
    
    dataX, dataY = [], []
    for i in tqdm( range(0, df.shape[0] + 1  - Lag - Horizon, overlap) ):
        
        # Original data
        data       = df.to_numpy()[i+1:(i+Lag)]        
        
        # Include date info
        if (includeDate):            
            myDate = [ np.sin(df[i+1:(i+Lag)].index.hour + df[i+1:(i+Lag)].index.minute / 60 ).tolist(), 
                       np.cos(df[i+1:(i+Lag)].index.hour + df[i+1:(i+Lag)].index.minute / 60 ).tolist() ]
            myDate = np.asarray( myDate ).T
            #
            # Concatenate data            
            data = np.concatenate([data, myDate], axis = -1)

        # data_diff: Lag-d differences
        if (includeLagData):
            data_diff  = (df.iloc[i+Lag-1] - df.iloc[i:(i+Lag-1)]).to_numpy()
            #
            # Concatenate data
            data = np.concatenate([data, data_diff], axis = -1)
                
        
        # Concatenate data
        dataX.append( data )
        
        # Targets 
        dataY.append( df[ targetSeries ].to_numpy()[i + Lag : i + Lag + Horizon] )
        
        
    return ( np.array(dataX), np.array(dataY) )

In [ ]:
trainX, trainY = create_dataset(df             = df_train, 
                                Lag            = Lag, 
                                Horizon        = Horizon, 
                                targetSeries   = targetSeries,
                                includeLagData = True,
                                includeDate    = False,
                                overlap        = 1)
                               

testX,  testY  = create_dataset(df             = df_test, 
                                Lag            = Lag, 
                                Horizon        = Horizon, 
                                targetSeries   = targetSeries,
                                includeLagData = True,
                                includeDate    = False,
                                overlap        = Horizon)


print('Training instances:   %6i' % trainX.shape[0])
print('Testing instances:    %6i' % testX.shape[0] )

# Hyperparameter optimization

In [ ]:
from   tensorflow.keras.layers                  import *
from   tensorflow.keras.models                  import *


def getModel( trainX, params):

    # Inputs
    #
    Inputs = Input(shape = (trainX.shape[1], trainX.shape[2]))


    # Encoder
    #
    Encoder = LSTM(units = params['units'], activation=params['activation_function'])( Inputs )

    if (params['Dropout_rate'] > 0):
        Encoder = Dropout( params['Dropout_rate'] )( Encoder )

    # Decoder
    #
    Decoder = RepeatVector( Horizon )(Encoder)
    #
    Decoder = LSTM(units = params['units'], activation=params['activation_function'])( Decoder )
    #
    # Output
    #
    Outputs = Dense( Horizon, activation='linear' )( Decoder )


    # Create model
    #
    model = Model(inputs = Inputs, outputs = Outputs)

    return ( model )

In [ ]:
class Parameter_Evaluation():
    def __init__(self, trainX, testX, trainY, testY, verbose = False):
        # Training/Testing sets
        #
        self.trainX = trainX
        self.testX  = testX
        self.trainY = trainY
        self.testY  = testY
        
        # Parameters
        self.verbose = verbose
        
        # Number of iterations
        #
        self.Iter   = 1
        
        # Best score - initialize
        #
        self.best_score = np.Inf
        
        # Hyperparameter optimization performance
        #
        self.hiplot_data = []
        

        
    def evaluate_params(self, params):
        
        # Store parameters
        #
        self.hiplot_data.append(params)

        
        # Setup model
        #
        model = getModel(self.trainX, params)
        
        # Define metrics
        #
        metrics = [
                    MeanAbsolutePercentageError(name = "MAPE", dtype = None),
                    RootMeanSquaredError(name = 'RMSE', dtype = None),
                  ]


        model.compile(loss      = params['loss'], 
                      optimizer = Adam(learning_rate = params['lr']), 
                      metrics   = metrics)

        # Checkpoint
        #
        checkpoint = ModelCheckpoint("checkpoints/Optimized_model_Hybrid.hdf5", 
                                      monitor        = 'val_RMSE', 
                                      verbose        = 0, 
                                      save_best_only = True, 
                                      mode           = 'min')

        # Earlystopping
        #
        earlystopping = EarlyStopping(monitor       = 'val_RMSE', 
                                      mode          = 'min', 
                                      verbose       = False, 
                                      patience      = 20)

        # Learning rate adjustment
        #
        lrs_scheduler  = ReduceLROnPlateau(monitor     = 'val_RMSE', 
                                           factor      = 0.5,
                                           patience    = 10)

        # Start clock
        #
        start = time.time()
        
        # Train model
        #
        model.fit(self.trainX, self.trainY,        
                  epochs           = epochs, 
                  batch_size       = params['batch_size'], 
                  callbacks        = [checkpoint, earlystopping, lrs_scheduler],
                  verbose          = False,#self.verbose, 
                  validation_split = 0.2)

        # Terminate clock
        #
        stop = time.time()
        if (self.verbose):
            print('[INFO] Model trained - Time %.2f' % (stop - start))


        # Load the best model
        #
        model.load_weights('checkpoints/Optimized_model_Hybrid.hdf5')

        # Evaluation
        #
        pred        = model.predict( self.trainX )
        train_score = rmse(self.trainY, pred)
        
        pred        = model.predict( self.testX )
        test_score  = rmse(self.testY, pred)
        
        # Check if test score is valid
        if (np.isnan(test_score)): test_score = 1000
            
        # Export results
        if (test_score < self.best_score):
            print("Iteration: {:3.0f} - RMSE = {:6.3f}/{:6.3f} (train/test) at {} (best)".format(self.Iter, train_score, test_score, str(datetime.now().time())[:8]))
            self.best_score = test_score
        else:
            print("Iteration: {:3.0f} - RMSE = {:6.3f}/{:6.3f} (train/test) at {}".format(self.Iter, train_score, test_score, str(datetime.now().time())[:8]))
        
        # Update Iteration counter
        self.Iter += 1

        # Store score
        self.hiplot_data[-1]['Score'] = test_score
        
        return( test_score )

In [ ]:
evaluator = Parameter_Evaluation(trainX  = trainX, 
                                 testX   = testX, 
                                 trainY  = trainY, 
                                 testY   = testY, 
                                 verbose = True)

# Prediction model

## Parameters

In [ ]:
search_space = [ 
                 skopt.space.Categorical(categories = ['relu', 'elu', 'swish', 'gelu'], 
                                         name="activation_function"),
                 skopt.space.Categorical(categories = [32, 64], 
                                         name="batch_size"),
                 skopt.space.Categorical(categories = [0.0, 0.2, 0.5], 
                                         name="Dropout_rate"),
                 skopt.space.Categorical(categories = [50, 100], 
                                         name="units"),
                 skopt.space.Categorical(categories = [1e-3, 5e-4, 1e-4, 1e-5], 
                                         name="lr"),
                 skopt.space.Categorical(categories = ['mse'], 
                                         name="loss"),
               ]

In [ ]:
HPO_params = {
              'n_calls':         n_calls,
              'n_random_starts': n_random_starts,
              'base_estimator':  'ET',
              'acq_func':        'EI',
             }

## Hyperparameter optimization process

In [ ]:
@skopt.utils.use_named_args( search_space )
def objective( **params ):
    return  evaluator.evaluate_params( params )

In [ ]:
%%time
results = skopt.forest_minimize(objective, search_space, **HPO_params)

### Plot results - Convergence

In [ ]:
plt.figure( figsize = (20, 5) )

skopt.plots.plot_convergence( results );

### Plot results - Evaluations

In [ ]:
# skopt.plots.plot_evaluations(results);

In [ ]:
# skopt.plots.plot_objective(results)

### Interactive parallel coordinates plots

In [ ]:
def np_encoder(object):
    if isinstance(object, np.generic):
        return object.item()

# Convert NumPy data types to be recognizable by json
evaluator.hiplot_data = json.loads( json.dumps(evaluator.hiplot_data, default=np_encoder) )

In [ ]:
hip.Experiment.from_iterable( evaluator.hiplot_data ).display();

### Save results

In [ ]:
with open('checkpoints/Hyperparameter_optimization_Hybrid.json', 'w', encoding='utf-8') as f:
    f.write( json.dumps( evaluator.hiplot_data ) )

## Get optimized hyperparameters

In [ ]:
def to_named_params(results, search_space):
    params       = results.x
    param_dict   = {}
    
    params_list  =[(dimension.name, param) for dimension, param in zip(search_space, params)]
    
    for item in params_list:
        param_dict[item[0]] = item[1]
    
    return( param_dict )

In [ ]:
optimized_params = to_named_params(results, search_space)


print('[INFO] Optimized hyperparameters\n')
#
for (parameter,value) in optimized_params.items():
    if ( isinstance(value, float) ):
        print(' >%25s: %.5f' % (parameter,value))
    else:
        print(' >%25s: %s' % (parameter,value))

# Optimized (best) model setup

In [ ]:
# Setup model
#
model = getModel(trainX, optimized_params)


print('[INFO] Model trained')

## Setup training parameters

In [ ]:
# Define metrics
#
metrics = [
            MeanAbsolutePercentageError(name = "MAPE", dtype = None),
            RootMeanSquaredError(name = 'RMSE', dtype = None),
          ]


model.compile(loss      = optimized_params['loss'], 
              optimizer = Adam(learning_rate = optimized_params['lr']), 
              metrics   = metrics)


# Checkpoint
#
checkpoint = ModelCheckpoint("checkpoints/Optimized_model_Hybrid.hdf5", 
                              monitor        = 'val_RMSE', 
                              verbose        = 0, 
                              save_best_only = True, 
                              mode           = 'min')

# Earlystopping
#
earlystopping = EarlyStopping(monitor       = 'val_RMSE', 
                              mode          = 'min', 
                              verbose       = 1, 
                              patience      = 20)

# Learning rate adjustment
#
lrs_scheduler  = ReduceLROnPlateau(monitor     = 'val_RMSE', 
                                   factor      = 0.5,
                                   patience    = 10)



## Train model

In [ ]:
# Start clock
#
start = time.time()


score = model.fit(trainX, trainY, 
                  epochs           = epochs, 
                  batch_size       = optimized_params['batch_size'], 
                  callbacks        = [checkpoint, earlystopping, lrs_scheduler],
                  verbose          = True, 
                  validation_split = 0.2)



# Terminate clock
#
stop = time.time()
print('[INFO] Time %.2f' % (stop - start))

## Training performance

In [ ]:
# Create figure
#
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (20, 3) )

# Plot RMSE
#
ax[0].plot( score.history['RMSE'], marker = 'o');
ax[0].plot( score.history['val_RMSE'], marker = 'o');
ax[0].legend(['Training', 'Validation'], frameon = False);
ax[0].set_title('RMSE');

# Plot RMSE
#
ax[1].plot( score.history['MAPE'], marker = 'o');
ax[1].plot( score.history['val_MAPE'], marker = 'o');
ax[1].legend(['Training', 'Validation'], frameon = False);
ax[1].set_title('MAPE');

## Evaluation

### Load optimized model

In [ ]:
# Load the best model
#
model.load_weights('checkpoints/Optimized_model_Hybrid.hdf5')

print('[INFO] Model loaded')

### Get predictions

In [ ]:
# Get predictions
#
pred = model.predict( testX )

### Apply inverse scaling/transformation

In [ ]:
# Apply inverse scaling
#
for i in range( Horizon ):
    testY[:,  i] = scaler.inverse_transform( testY[:,  i].reshape(-1,1) ).squeeze(-1)
    pred[:, i]   = scaler.inverse_transform( pred[:, i].reshape(-1,1) ).squeeze(-1)


# Apply inverse transformation   
#
if (Transformation == True):
    testY = np.exp( testY ) - VALUE
    pred = np.exp( pred )   - VALUE

### Calculate Performance on Testing set

In [ ]:
print('[INFO] Feature: ', targetSeries)
print('------------------------------------------------')
Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

for i in range( Horizon ):

    Prices = pd.DataFrame([])        

    Prices[targetSeries] = testY[:,i]
    Prices['Prediction'] = pred[:,i]


    # Evaluation
    #
    MAE, RMSE, MAPE, SMAPE, R2 = RegressionEvaluation( Prices )

    # Store results
    #
    Performance_Foresting_Model['RMSE']    += [ RMSE    ]
    Performance_Foresting_Model['MAE']     += [ MAE     ]
    Performance_Foresting_Model['SMAPE']   += [ SMAPE   ]
    Performance_Foresting_Model['R2']      += [ R2      ]

    # Present results
    #
#     print('Horizon: ', i+1)
#     print('> MAE:   ', MAE)
#     print('> RMSE:  ', RMSE)
#     print('> SMAPE: ', SMAPE)
#     print('> R2:    ', R2)
#     print()
    
    print('Horizon: %2i MAE %5.2f SMAPE: %5.2f R2: %.2f' % (i+1, MAE, SMAPE, R2) )

### Reliability evaluation

In [ ]:
print('[INFO] Feature: ', targetSeries)
print('------------------------------------------------')
Performance_Foresting_Model = {'RMSE': [], 'MAE': [], 'SMAPE': [], 'R2' : []}

for i in range( Horizon ):

    Prices = pd.DataFrame([])        

    Prices[targetSeries] = testY[:,i]
    Prices['Prediction'] = pred[:,i]
            
            

            
    from scipy import stats
    from statsmodels.graphics.tsaplots import plot_acf
    
    res = (Prices[targetSeries] - Prices['Prediction']).to_numpy()
    
    fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (15, 2) )

    ax[0].hist( res, bins = 100 )    
    plot_acf( res, ax=ax[1] )       

## Visualization

In [ ]:
Prices = pd.DataFrame( [] )

Prices[targetSeries] = testY.flatten()
Prices['Prediction'] = pred.flatten()


# Plot results
#
plt.figure( figsize = (20, 4) );
#
Prices[ targetSeries ][:200].plot(color = 'tab:blue', marker = 'o')
Prices[ 'Prediction' ][:200].plot(color = 'tab:orange', marker = 'o')
plt.show();


### Examples

In [ ]:
subplots = [331, 332, 333, 334, 335, 336,  337, 338, 339]
plt.figure( figsize = (20, 8) )
RandomInstances = [random.randint(1, testY.shape[0]) for i in range(0, 9)]
RandomInstances = [66, 62, 192, 448, 477, 1035, 34, 408, 860]

for plot_id, i in enumerate(RandomInstances):

    plt.subplot(subplots[plot_id])
    plt.grid()

    plt.plot(testY[i], color='g', marker = 'o', linewidth = 2)
    plt.plot(pred[i],  color='r', marker = 'o', linewidth = 2)

    plt.legend(['Future values', 'Prediction'], frameon = False, fontsize = 12)
    plt.ylim([90, 260])
plt.show()